# Code source : machinelearningmystry.com

#                         1-Univariate Time Seies

In [1]:
# import libraries
import numpy as np
from numpy import array
from numpy import hstack
import pandas as pd
import tensorflow as tf # TENSORFLOW V2
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.layers import LSTM, Dense, Dropout

# old method

don't use it ( it is not including batch size ), use time series genrator instead

In [1]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[10 20 30] 40
[20 30 40] 50
[30 40 50] 60
[40 50 60] 70
[50 60 70] 80
[60 70 80] 90


# using timeseriesgenerator

In [22]:
# define dataset
series = array([10, 20, 30, 40, 50, 60, 70, 80, 90])

In [24]:
# define generator
n_input = 3                      #as X , as y
batch_s = 1
generator = TimeseriesGenerator(series, series, length=n_input, batch_size=batch_s)

In [25]:
# number of samples
print('Samples: %d' % len(generator))
# print each sample
for i in range(len(generator)):
	x, y = generator[i]
	print('%s %s' % (x, y))

Samples: 6
[[10 20 30]] [40]
[[20 30 40]] [50]
[[30 40 50]] [60]
[[40 50 60]] [70]
[[50 60 70]] [80]
[[60 70 80]] [90]


# MLP Model

A simple MLP model has a single hidden layer of nodes, and an output layer used to make a prediction.

In [26]:
# import libraries
import tensorflow as tf
from tensorflow.keras.layers import Dense

In [27]:
# define model
model = tf.keras.Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [28]:
# fit model
model.fit(x, y, epochs=2000, verbose=0)

In [29]:
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)

In [30]:
yhat

array([[102.88401]], dtype=float32)

In [13]:
# univariate mlp example
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = [10, 20, 30, 40, 50, 60, 70, 80, 90]
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# define model
model = tf.keras.Sequential()
model.add(Dense(100, activation='relu', input_dim=n_steps))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=2000, verbose=0)
# demonstrate prediction
x_input = array([70, 80, 90])
x_input = x_input.reshape((1, n_steps))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[100.1941]]


In [32]:
# univariate one step problem with mlp
# define dataset
series2 = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# define generator
n_input = 2
generator = TimeseriesGenerator(series2, series2, length=n_input, batch_size=8)
# define model
model = tf.keras.Sequential()
model.add(Dense(100, activation='relu', input_dim=n_input))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit_generator(generator, steps_per_epoch=1, epochs=200, verbose=0)
# make a one step prediction out of sample
x_input = array([9, 10]).reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[11.412065]]


In [3]:
# define dataset
series5 = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
# reshape to [10, 1]
n_features = 1
series5 = series5.reshape((len(series5), n_features))
# define generator
n_input = 2
generator = TimeseriesGenerator(series5, series5, length=n_input, batch_size=8)
# define model
model = tf.keras.Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit_generator(generator, epochs=500, verbose=0)
# make a one step prediction out of sample
x_input = array([9 , 10]).reshape((1, n_input, n_features)).astype(np.float32)
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[11.047657]]


In [4]:
series5.dtype

dtype('int32')

In [5]:
x_input.dtype

dtype('float32')

# Multivariate Time Series Example

In [2]:
# define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])

In [3]:
# reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))

In [4]:
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2))

In [5]:
# define generator
# define generator
n_features = dataset.shape[1]
n_input = 2
generator = TimeseriesGenerator(dataset, dataset, length=n_input, batch_size=1)

Each sample will then be a three-dimensional array of [1, 2, 2] for the 1 sample, 2 time steps, and 2 features or parallel series. The output will be a two-dimensional series of [1, 2] for the 1 sample and 2 features. The first sample will be:

In [6]:
# number of samples
print('Samples: %d' % len(generator))
# print each sample
for i in range(len(generator)):
	x, y = generator[i]
	print('%s => %s' % (x, y))

Samples: 8
[[[10 15]
  [20 25]]] => [[30 35]]
[[[20 25]
  [30 35]]] => [[40 45]]
[[[30 35]
  [40 45]]] => [[50 55]]
[[[40 45]
  [50 55]]] => [[60 65]]
[[[50 55]
  [60 65]]] => [[70 75]]
[[[60 65]
  [70 75]]] => [[80 85]]
[[[70 75]
  [80 85]]] => [[90 95]]
[[[80 85]
  [90 95]]] => [[100 105]]


In [10]:
# define model
model = tf.keras.Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_input, n_features)))
model.add(Dense(2))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit_generator(generator, steps_per_epoch=1, epochs=500, verbose=0)
# make a one step prediction out of sample
x_input = array([[90, 95], [100, 105]]).reshape((1, n_input, n_features)).astype(np.float32)
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[111.51708 117.08021]]


# Multivariate Inputs and Dependent Series Example

In [111]:
# define dataset
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95, 105])
out_seq = array([25, 45, 65, 85, 105, 125, 145, 165, 185, 205])

In [112]:
# reshape series
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2))
# define generator
n_input = 1
generator = TimeseriesGenerator(dataset, out_seq, length=n_input, batch_size=1)
# print each sample
for i in range(len(generator)):
	x, y = generator[i]
	print('%s => %s' % (x, y))

[[[10 15]]] => [[45]]
[[[20 25]]] => [[65]]
[[[30 35]]] => [[85]]
[[[40 45]]] => [[105]]
[[[50 55]]] => [[125]]
[[[60 65]]] => [[145]]
[[[70 75]]] => [[165]]
[[[80 85]]] => [[185]]
[[[90 95]]] => [[205]]


# Multi-step Forecasts Example

A benefit of neural network models over many other types of classical and machine learning models is that they can make multi-step forecasts.

That is, that the model can learn to map an input pattern of one or more features to an output pattern of more than one feature. This can be used in time series forecasting to directly forecast multiple future time steps.

This can be achieved either by directly outputting a vector from the model, by specifying the desired number of outputs as the number of nodes in the output layer, or it can be achieved by specialized sequence prediction models such as an encoder-decoder model.

A limitation of the TimeseriesGenerator is that it does not directly support multi-step outputs. Specifically, it will not create the multiple steps that may be required in the target sequence.

Nevertheless, if you prepare your target sequence to have multiple steps, it will honor and use them as the output portion of each sample. This means the onus is on you to prepare the expected output for each time step.

We can demonstrate this with a simple univariate time series with two time steps in the output sequence.

You can see that you must have the same number of rows in the target sequence as you do in the input sequence. In this case, we must know values beyond the values in the input sequence, or trim the input sequence to the length of the target sequence.

In [51]:
# define dataset
series = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
target = array([[1,2],[2,3],[3,4],[4,5],[5,6],[6,7],[7,8],[8,9],[9,10],[10,11]])

In [53]:
# define generator
n_input = 3
generator = TimeseriesGenerator(series, target, length=n_input, batch_size=1)
# print each sample
for i in range(len(generator)):
	x, y = generator[i]
	print('%s => %s' % (x, y))

[[1 2 3]] => [[4 5]]
[[2 3 4]] => [[5 6]]
[[3 4 5]] => [[6 7]]
[[4 5 6]] => [[7 8]]
[[5 6 7]] => [[8 9]]
[[6 7 8]] => [[ 9 10]]
[[7 8 9]] => [[10 11]]


# DataFrame

In [132]:
X1 = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]
X2 = [15, 25, 35, 45, 55, 65, 75, 85, 95, 105]
y = [ 25 ,45, 65, 85, 105, 125, 145, 165, 185, 205]

In [143]:
df = pd.DataFrame(list(zip(X1,X2,y)),columns=['X1','X2','y'])

In [144]:
df

,X1,X2,y
0,10,15,25
1,20,25,45
2,30,35,65
3,40,45,85
4,50,55,105
5,60,65,125
6,70,75,145
7,80,85,165
8,90,95,185
9,100,105,205


In [145]:
df['y'] = df['y'].shift(-1)

In [146]:
df

,X1,X2,y
0,10,15,45.0
1,20,25,65.0
2,30,35,85.0
3,40,45,105.0
4,50,55,125.0
5,60,65,145.0
6,70,75,165.0
7,80,85,185.0
8,90,95,205.0
9,100,105,NaN


In [147]:
# drop the row with nan
df = df.dropna()

In [148]:
df

,X1,X2,y
0,10,15,45.0
1,20,25,65.0
2,30,35,85.0
3,40,45,105.0
4,50,55,125.0
5,60,65,145.0
6,70,75,165.0
7,80,85,185.0
8,90,95,205.0


In [150]:
# convert dataframe into numpy array
df = df.values

In [151]:
df

array([[ 10.,  15.,  45.],
       [ 20.,  25.,  65.],
       [ 30.,  35.,  85.],
       [ 40.,  45., 105.],
       [ 50.,  55., 125.],
       [ 60.,  65., 145.],
       [ 70.,  75., 165.],
       [ 80.,  85., 185.],
       [ 90.,  95., 205.]])

In [156]:
# split input and output
X , y = df[:,:-1],df[:,-1]

In [157]:
X

array([[10., 15.],
       [20., 25.],
       [30., 35.],
       [40., 45.],
       [50., 55.],
       [60., 65.],
       [70., 75.],
       [80., 85.],
       [90., 95.]])

In [158]:
y

array([ 45.,  65.,  85., 105., 125., 145., 165., 185., 205.])

In [ ]:
# now data is suitable for most of prediction algorithms
# finnaly after bulding prediction model we can add the row that we had dropped it to (predict) it.